In [ ]:

!mkdir -p /content/drive/MyDrive/medical-image-caption-data/

%cd /content/drive/MyDrive/


!tar -xvzf NLMCXR_png.tgz -C ./medical-image-caption-data/


!tar -xvzf NLMCXR_reports.tgz -C ./medical-image-caption-data/


Streaming output truncated to the last 5000 lines.
./CXR3471_IM-1687-2002.png
./CXR3810_IM-1920-2001.png
./CXR122_IM-0147-0001-0001.png
./CXR2927_IM-1329-1001.png
./CXR594_IM-2187-2001.png
./CXR683_IM-2254-1001.png
./CXR234_IM-0906-0001-0001.png
./CXR3292_IM-1572-1001.png
./CXR81_IM-2343-2001.png
./CXR1347_IM-0225-3001.png
./CXR2543_IM-1054-3001.png
./CXR3315_IM-1586-2001.png
./CXR2471_IM-1002-1001.png
./CXR3468_IM-1684-0001-0004.png
./CXR3060_IM-1426-1003.png
./CXR57_IM-2170-1001-0002.png
./CXR3858_IM-1953-4004.png
./CXR1541_IM-0352-2001.png
./CXR2845_IM-1254-2001.png
./CXR2549_IM-1057-1001.png
./CXR210_IM-0730-1001.png
./CXR2117_IM-0745-1001.png
./CXR3978_IM-2037-0001-0002.png
./CXR3881_IM-1969-1001.png
./CXR3854_IM-1950-2001.png
./CXR2195_IM-0805-2001.png
./CXR439_IM-2078-2001.png
./CXR1280_IM-0187-1001.png
./CXR666_IM-2241-1001.png
./CXR2804_IM-1235-1001.png
./CXR3783_IM-1898-1001.png
./CXR3533_IM-1726-2001.png
./CXR516_IM-2130-2001.png
./CXR379_IM-1903-2001.png
./CXR2100_IM-0731-2

## CONNECT TO MY GOOGLE DRIVE

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:

image_dir = '/content/drive/MyDrive/medical-image-caption-data'
report_dir = '/content/drive/MyDrive/medical-report'


In [4]:
import os
print("Number of images:", len(os.listdir(image_dir)))
print("Number of reports:", len(os.listdir(report_dir)))

Number of images: 2065
Number of reports: 3955


## import the libraries

In [5]:
import os
from PIL import Image
import xml.etree.ElementTree as ET
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import AutoTokenizer
from tqdm import tqdm
import torch.nn as nn


## Pre-process image

In [6]:
from torchvision import transforms

def get_image_transform(train=True):
    if train:
        # data enhancement in training
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.RandomHorizontalFlip(p=0.5),
            transforms.RandomRotation(degrees=10),
            transforms.ColorJitter(brightness=0.2, contrast=0.2),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    else:
        # testing/evaluate progress
        transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])
    return transform


In [7]:

train_transform = get_image_transform(train=True)

val_transform = get_image_transform(train=False)


## Initialize Tokenizer --- BERT --- Compare to the T-5 smaller later


In [9]:
from transformers import AutoTokenizer

# 加载 HuggingFace 的 tokenizer（选用 BERT）
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

## Customize the dataset class

In [10]:
from PIL import Image
class IUXRayDataset(Dataset):
    def __init__(self, image_dir, report_dir, tokenizer, max_length=100):
        self.image_dir = image_dir
        self.report_dir = report_dir
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_samples()

        # process image
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    def load_samples(self):
        print("===> Start loading samples from report and image...")
        samples = []
        count_img = 0
        count_match = 0
        count_missing_report = 0

        for img_name in tqdm(os.listdir(self.image_dir), desc="Parsing image-report pairs"):
            if not img_name.endswith('.png'):
                continue

            count_img += 1

            # extract the id
            base_id = img_name.split('_')[0]  # CXR1005
            report_id = base_id.replace('CXR', '')  # 1005

            # do the xml path
            xml_path = os.path.join(self.report_dir, report_id + ".xml")

            if not os.path.exists(xml_path):
                count_missing_report += 1
                continue

            # load the  xml
            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
            except:
                continue

            findings, impression = "", ""
            for abstract in root.findall('.//AbstractText'):
                label = abstract.attrib.get('Label')
                if label == 'FINDINGS':
                    findings = abstract.text or ""
                if label == 'IMPRESSION':
                    impression = abstract.text or ""

            text = (findings + " " + impression).strip().replace("\n", " ")
            if not text:
                continue

            samples.append((img_name, text))
            count_match += 1

        print(f"✅ Total images scanned: {count_img}")
        print(f"✅ Total matched samples with report: {count_match}")
        print(f"⚠️  Total images missing report: {count_missing_report}")
        return samples



    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name, text = self.samples[idx]
        img_path = os.path.join(self.image_dir, img_name)


        image = Image.open(img_path).convert("RGB")

        # transform to Tensor
        image = self.transform(image)

        tokens = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'image': image,  # Tensor
            'input_ids': tokens.input_ids.squeeze(0),
            'attention_mask': tokens.attention_mask.squeeze(0)
        }

## Create Dataset and DataLoader

In [11]:
# create the complete  Dataset

full_dataset = IUXRayDataset(
    image_dir=image_dir,
    report_dir=report_dir,
    tokenizer=tokenizer,
    max_length=100  # control the length of tokenizer
)


===> Start loading samples from report and image...


Parsing image-report pairs: 100%|██████████| 2065/2065 [01:19<00:00, 25.89it/s] 

✅ Total images scanned: 2063
✅ Total matched samples with report: 2054
⚠️  Total images missing report: 0


In [12]:
print(full_dataset[0])

{'image': tensor([[[0.2000, 0.1882, 0.1843,  ..., 0.7843, 0.7922, 0.7922],
         [0.1922, 0.1922, 0.3059,  ..., 0.7882, 0.7843, 0.7843],
         [0.1922, 0.2000, 0.5882,  ..., 0.7804, 0.7882, 0.7804],
         ...,
         [0.7608, 0.7529, 0.7529,  ..., 0.3490, 0.3490, 0.3569],
         [0.7647, 0.7647, 0.7647,  ..., 0.4314, 0.4392, 0.4471],
         [0.7725, 0.7647, 0.7647,  ..., 0.6824, 0.6902, 0.7059]],

        [[0.2000, 0.1882, 0.1843,  ..., 0.7843, 0.7922, 0.7922],
         [0.1922, 0.1922, 0.3059,  ..., 0.7882, 0.7843, 0.7843],
         [0.1922, 0.2000, 0.5882,  ..., 0.7804, 0.7882, 0.7804],
         ...,
         [0.7608, 0.7529, 0.7529,  ..., 0.3490, 0.3490, 0.3569],
         [0.7647, 0.7647, 0.7647,  ..., 0.4314, 0.4392, 0.4471],
         [0.7725, 0.7647, 0.7647,  ..., 0.6824, 0.6902, 0.7059]],

        [[0.2000, 0.1882, 0.1843,  ..., 0.7843, 0.7922, 0.7922],
         [0.1922, 0.1922, 0.3059,  ..., 0.7882, 0.7843, 0.7843],
         [0.1922, 0.2000, 0.5882,  ..., 0.7804, 

##Summary:
every image has a report, but not every report can find a matching image

## Test the dataloader and pair the image and report


In [13]:
import torch
from torch.utils.data import Subset

# subtract the first 10 samples
small_dataset = Subset(full_dataset, list(range(10)))

print(f"Amount of small samples: {len(small_dataset)}")




# built the  DataLoader
small_loader = DataLoader(small_dataset, batch_size=2, shuffle=True)

# testing
for batch in small_loader:
    print(batch['image'].shape)          # ✅ [B, 3, H, W]
    print(batch['input_ids'].shape)      # ✅ [B, T]
    print(batch['attention_mask'].shape) # ✅ [B, T]
    break


Amount of small samples: 10
torch.Size([2, 3, 224, 224])
torch.Size([2, 100])
torch.Size([2, 100])


# Training DataLoader & validation dataloader

In [15]:


train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"Training dataset: {len(train_dataset)}")
print(f"Testing Dataset: {len(val_dataset)}")

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Training dataset: 1643
Testing Dataset: 411


## Initialize the model
## Encoder & Decoder

In [37]:
from transformers import CLIPModel, CLIPProcessor

class EncoderCLIP(nn.Module):
    def __init__(self, embed_size, device):
        super(EncoderCLIP, self).__init__()
        self.device = device
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


        self.fc = nn.Linear(self.clip_model.config.projection_dim, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

    def forward(self, images):

        inputs = self.processor(images=images, return_tensors="pt").to(self.device)
        image_features = self.clip_model.get_image_features(**inputs)  # [batch, 512]
        image_features = self.fc(image_features)                       # [batch, embed_size]
        image_features = self.bn(image_features)




        return image_features


In [38]:
class DecoderTransformer(nn.Module):
    def __init__(self, embed_size, vocab_size, num_layers=4, heads=4, ff_hidden=1024, dropout=0.3, max_len=100):
        super(DecoderTransformer, self).__init__()

        #  token embedding
        self.embed = nn.Embedding(vocab_size, embed_size)


        self.positional_encoding = PositionalEncoding(embed_size, dropout, max_len)


        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_size,
            nhead=heads,
            dim_feedforward=ff_hidden,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)

        #  vocab size
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, image_features, captions):

        embeddings = self.embed(captions)
        embeddings = self.positional_encoding(embeddings)


        tgt_mask = self.generate_square_subsequent_mask(captions.size(1)).to(captions.device)


        output = self.transformer_decoder(
            tgt=embeddings,
            memory=image_features.unsqueeze(1),
            tgt_mask=tgt_mask
        )


        output = self.fc(output)
        return output

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


# 位置编码模块（经典做法）
class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)  # [1, max_len, embed]
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


In [39]:


class ImageCaptionModel(nn.Module):
    def __init__(self, embed_size, vocab_size, device):
        super(ImageCaptionModel, self).__init__()
        self.encoder = EncoderCLIP(embed_size, device)
        self.decoder = DecoderTransformer(embed_size, vocab_size)

    def forward(self, images, captions):
        image_features = self.encoder(images)  # [batch, embed]
        outputs = self.decoder(image_features, captions)
        return outputs


In [40]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageCaptionModel(embed_size=512, vocab_size=len(tokenizer), device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

# EXP01

### TRAIN IN A LOOP

In [16]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

def train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=10, save_path="checkpoint.pth"):
    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        total_loss = 0
        loop = tqdm(train_loader, leave=True, dynamic_ncols=True)
        for batch in loop:
            images = batch['image'].to(device)
            captions = batch['input_ids'].to(device)

            optimizer.zero_grad()
            outputs = model(images, captions[:, :-1])
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), captions[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

        # save the latest checkpoint for every epoch
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }

        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        torch.save(checkpoint, save_path)
        print(f"Checkpoint saved at {save_path}")


In [19]:
train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=10,save_path="/content/EXP01-checkpoint.pth")

Epoch [1/10]: 100%|██████████| 52/52 [09:03<00:00, 10.45s/it, loss=4.98]


Epoch 1, Loss: 329.2802
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [2/10]: 100%|██████████| 52/52 [00:46<00:00,  1.11it/s, loss=3.42]


Epoch 2, Loss: 205.4930
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [3/10]: 100%|██████████| 52/52 [00:47<00:00,  1.10it/s, loss=3.55]


...

Epoch 8, Loss: 107.6193
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [9/10]: 100%|██████████| 52/52 [00:47<00:00,  1.09it/s, loss=1.85]


Epoch 9, Loss: 103.0175
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [10/10]: 100%|██████████| 52/52 [00:47<00:00,  1.09it/s, loss=1.85]


Epoch 10, Loss: 99.0398
Checkpoint saved at /content/EXP01-checkpoint.pth


In [17]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

def train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=10, save_path="checkpoint.pth"):
    model.to(device)

    start_epoch = 0


    if os.path.exists(save_path):
        print(f"===> Found checkpoint at {save_path}, loading ...")
        checkpoint = torch.load(save_path, map_location=device)

        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1

        print(f"===> Resume training from epoch {start_epoch}")

    model.train()

    for epoch in range(start_epoch, num_epochs):
        total_loss = 0
        loop = tqdm(train_loader, leave=True)

        for batch in loop:
            images = batch['image'].to(device)
            captions = batch['input_ids'].to(device)

            optimizer.zero_grad()
            outputs = model(images, captions[:, :-1])
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), captions[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }

        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        torch.save(checkpoint, save_path)
        print(f"Checkpoint saved at {save_path}")


In [21]:
train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=463,save_path="/content/EXP01-checkpoint.pth")

===> Found checkpoint at /content/EXP01-checkpoint.pth, loading ...
===> Resume training from epoch 400


Epoch [401/700]: 100%|██████████| 52/52 [00:47<00:00,  1.09it/s, loss=0.133]


Epoch 401, Loss: 7.7084
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [402/700]: 100%|██████████| 52/52 [00:47<00:00,  1.09it/s, loss=0.139]


Epoch 402, Loss: 7.7246
Checkpoint saved at /content/EXP01-checkpoint.pth


...

Epoch [460/700]: 100%|██████████| 52/52 [00:47<00:00,  1.10it/s, loss=0.153]


Epoch 460, Loss: 8.2169
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [461/700]: 100%|██████████| 52/52 [00:47<00:00,  1.10it/s, loss=0.186]


Epoch 461, Loss: 8.1197
Checkpoint saved at /content/EXP01-checkpoint.pth


Epoch [462/700]:  44%|████▍     | 23/52 [00:21<00:27,  1.06it/s, loss=0.135]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.


KeyboardInterrupt



## Evaluation

In [25]:
import torch
from tqdm import tqdm

def evaluate(model, test_loader, tokenizer, device, max_length=100, num_samples=5):
    model.eval()
    model.to(device)

    results = []

    eos_token_id = tokenizer.eos_token_id or tokenizer.sep_token_id or tokenizer.pad_token_id

    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_loader)):
            images = batch['image'].to(device)
            gt_reports = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)


            start_token = tokenizer.bos_token_id or tokenizer.cls_token_id or 0
            input_ids = torch.full((images.size(0), 1), start_token, dtype=torch.long).to(device)

            # greedy decoding
            for _ in range(max_length):
                outputs = model(images, input_ids)           # [B, T, V]
                next_token = outputs[:, -1, :].argmax(dim=-1, keepdim=True)  # [B, 1]
                input_ids = torch.cat([input_ids, next_token], dim=1)

                if eos_token_id is not None:
                    if torch.eq(next_token, eos_token_id).all():
                        break

            # decode
            generated_reports = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

            for gt, pred in zip(gt_reports, generated_reports):
                results.append((gt.strip(), pred.strip()))


            if len(results) >= num_samples:
                break

    print("\n=== Sample Evaluation ===")
    for idx, (gt, pred) in enumerate(results[:num_samples]):
        print(f"\n🔹 Sample {idx+1}")
        print(f"🟢 Ground Truth:\n{gt}")
        print(f"🔵 Prediction:\n{pred}")

    return results


In [26]:
print(f"EOS token ID: {tokenizer.eos_token_id}")

EOS token ID: None


In [27]:

evaluate(model, val_loader, tokenizer, device, num_samples=5)


  0%|          | 0/13 [00:25<?, ?it/s]


=== Sample Evaluation ===

🔹 Sample 1
🟢 Ground Truth:
the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. multilevel flowing anterior thoracic spine osteophytes, which could represent changes of diffuse idiopathic skeletal hyperostosis ( dish ). there is no evidence of acute cardiopulmonary disease.
🔵 Prediction:
media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media med

[('the cardiac silhouette and mediastinum size are within normal limits. there is no pulmonary edema. there is no focal consolidation. there are no xxxx of a pleural effusion. there is no evidence of pneumothorax. multilevel flowing anterior thoracic spine osteophytes, which could represent changes of diffuse idiopathic skeletal hyperostosis ( dish ). there is no evidence of acute cardiopulmonary disease.',
  'media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media media medi

# EXP02

## Encoder

In [51]:
from transformers import CLIPModel, CLIPProcessor


class EncoderCLIP(nn.Module):
    def __init__(self, embed_size, device):
        super(EncoderCLIP, self).__init__()
        self.device = device
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        self.fc = nn.Linear(768, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

    def forward(self, images):
        inputs = self.processor(images=images, return_tensors="pt").to(self.device)
        vision_outputs = self.clip_model.vision_model(**inputs)
        patch_embeddings = vision_outputs.last_hidden_state  # [B, seq_len, 768]

        projected = self.fc(patch_embeddings)  # → [B, seq_len, embed]
        projected = self.bn(projected.transpose(1, 2)).transpose(1, 2)

        return projected  # [B, seq_len, embed]



## Decoder

In [52]:
import torch
import torch.nn as nn

class DecoderTransformer(nn.Module):
    def __init__(self, embed_size, vocab_size, num_layers=4, heads=4, ff_hidden=1024, dropout=0.3, max_len=100):
        super(DecoderTransformer, self).__init__()

        self.embed = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size, dropout, max_len)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_size,
            nhead=heads,
            dim_feedforward=ff_hidden,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, image_features, captions):
        # captions: [B, T] (input_ids)
        embeddings = self.embed(captions)
        embeddings = self.positional_encoding(embeddings)

        tgt_mask = self.generate_square_subsequent_mask(captions.size(1)).to(captions.device)

        output = self.transformer_decoder(
            tgt=embeddings,
            memory=image_features,
            tgt_mask=tgt_mask
        )

        return self.fc(output)

    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


In [53]:
class ImageCaptionModel(nn.Module):
    def __init__(self, embed_size, vocab_size, device):
        super(ImageCaptionModel, self).__init__()
        self.encoder = EncoderCLIP(embed_size, device)
        self.decoder = DecoderTransformer(embed_size, vocab_size)

    def forward(self, images, captions):
        image_features = self.encoder(images)  # [batch, embed]
        outputs = self.decoder(image_features, captions)
        return outputs


## Initialize model

In [54]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageCaptionModel(embed_size=512, vocab_size=len(tokenizer), device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

## Train In a Loop

In [55]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

def train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=10, save_path="checkpoint.pth"):

    model.to(device)
    start_epoch = 0


    if os.path.exists(save_path):
        print(f"==> Found checkpoint at {save_path}, loading...")
        checkpoint = torch.load(save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"==> Resuming training from epoch {start_epoch}")
    model.train()

    for epoch in range(start_epoch,num_epochs):
        total_loss = 0
        loop = tqdm(train_loader, leave=True, dynamic_ncols=True)
        for batch in loop:
            images = batch['image'].to(device)
            captions = batch['input_ids'].to(device)

            optimizer.zero_grad()
            outputs = model(images, captions[:, :-1])
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), captions[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }


        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        torch.save(checkpoint, save_path)
        print(f"Checkpoint saved at {save_path}")


In [78]:
train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=100,save_path="/content/drive/MyDrive/EXP02-checkpoint.pth")

==> Found checkpoint at /content/EXP02-checkpoint.pth, loading...
==> Resuming training from epoch 10


Epoch [11/100]: 100%|██████████| 52/52 [00:48<00:00,  1.08it/s, loss=0.574]


Epoch 11, Loss: 28.9077
Checkpoint saved at /content/EXP02-checkpoint.pth


Epoch [12/100]: 100%|██████████| 52/52 [00:48<00:00,  1.08it/s, loss=0.395]


Epoch 12, Loss: 28.4044
Checkpoint saved at /content/EXP02-checkpoint.pth


...

Epoch 98, Loss: 17.0536
Checkpoint saved at /content/EXP02-checkpoint.pth


Epoch [99/100]: 100%|██████████| 52/52 [00:48<00:00,  1.08it/s, loss=0.334]


Epoch 99, Loss: 16.9223
Checkpoint saved at /content/EXP02-checkpoint.pth


Epoch [100/100]: 100%|██████████| 52/52 [00:48<00:00,  1.07it/s, loss=0.297]


Epoch 100, Loss: 16.8713
Checkpoint saved at /content/EXP02-checkpoint.pth


## Evaluation

In [40]:
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

def apply_repetition_penalty(logits, input_ids, penalty=1.2):
    """
    惩罚已生成的 token，防止重复生成同一个词。
    """
    for i in range(logits.size(0)):
        for token_id in input_ids[i].tolist():
            logits[i, token_id] /= penalty
    return logits


def evaluate(model, test_loader, tokenizer, device, max_length=100, num_samples=5,
             temperature=1.0, top_k=50, repetition_penalty=1.2):
    model.eval()
    model.to(device)

    eos_token_id = tokenizer.eos_token_id or tokenizer.sep_token_id or tokenizer.pad_token_id or 0
    bos_token_id = tokenizer.bos_token_id or tokenizer.cls_token_id or 0

    results = []

    with torch.no_grad():
        for i, batch in enumerate(tqdm(test_loader)):
            images = batch['image'].to(device)
            gt_reports = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)

            # init: [B, 1]
            input_ids = torch.full((images.size(0), 1), bos_token_id, dtype=torch.long).to(device)

            for _ in range(max_length):
                outputs = model(images, input_ids)        # [B, T, vocab]
                logits = outputs[:, -1, :]
                logits = logits / temperature

                logits = apply_repetition_penalty(logits, input_ids, penalty=repetition_penalty)

                # Top-k sampling
                values, indices = torch.topk(logits, k=top_k, dim=-1)      # [B, K]
                probs = torch.softmax(values, dim=-1)                      # [B, K]
                sampled = torch.multinomial(probs, num_samples=1)         # [B, 1]
                next_token = indices.gather(-1, sampled)                  # [B, 1]

                input_ids = torch.cat([input_ids, next_token], dim=1)

                if eos_token_id and torch.eq(next_token, eos_token_id).all():
                    break

            predictions = tokenizer.batch_decode(input_ids, skip_special_tokens=True)

            for gt, pred in zip(gt_reports, predictions):
                results.append((gt.strip(), pred.strip()))

            if len(results) >= num_samples:
                break


    print("\n=== Sample Evaluation ===")
    for idx, (gt, pred) in enumerate(results[:num_samples]):
        print(f"\n🔹 Sample {idx+1}")
        print(f"🟢 Ground Truth:\n{gt}")
        print(f"🔵 Prediction:\n{pred}")

    # BLEU score
    pred_tokens = [pred.split() for _, pred in results]
    gt_tokens = [[gt.split()] for gt, _ in results]
    bleu_score = corpus_bleu(gt_tokens, pred_tokens)
    print(f"\n🎯 BLEU Score: {bleu_score:.4f}")

    return results, bleu_score


In [41]:

# evaluate(model, val_loader, tokenizer, device, num_samples=5)
evaluate(
    model,
    val_loader,
    tokenizer,
    device,
    num_samples=5,
    temperature=1.0,
    top_k=50,
    repetition_penalty=1.2
)

  0%|          | 0/13 [00:38<?, ?it/s]


=== Sample Evaluation ===

🔹 Sample 1
🟢 Ground Truth:
HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: Mild cardiomegaly. Normal size and mediastinal contours. Clear lungs. No pneumothorax or pleural effusion. Unremarkable XXXX. IMPRESSION: Mild cardio
🔵 Prediction:
media to moderate images in size within normal limits. Vascular stable medial increased lung volumes with XX bron: N/A PNEUMOTHORAX: N/A FINDINGS: Stable appearance of the left base, scarring, and alvenous catheter tip overlieth atelectasis bilateral bibasilar airspace disease on the adjacent to prior study. No pneumothorax or large effusion. Left costoph

🔹 Sample 2
🟢 Ground Truth:
HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The cardiac and mediastinal contours are normal. The lungs are well-inflated and clear. There is no focal consolidation, pneumothorax or effusion. No acute bony abnormalities are seen. No radiopaque foreign bodies
🔵 Prediction:
superior.

([('HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: Mild cardiomegaly. Normal size and mediastinal contours. Clear lungs. No pneumothorax or pleural effusion. Unremarkable XXXX. IMPRESSION: Mild cardio',
   'media to moderate images in size within normal limits. Vascular stable medial increased lung volumes with XX bron: N/A PNEUMOTHORAX: N/A FINDINGS: Stable appearance of the left base, scarring, and alvenous catheter tip overlieth atelectasis bilateral bibasilar airspace disease on the adjacent to prior study. No pneumothorax or large effusion. Left costoph'),
  ('HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The cardiac and mediastinal contours are normal. The lungs are well-inflated and clear. There is no focal consolidation, pneumothorax or effusion. No acute bony abnormalities are seen. No radiopaque foreign bodies',
   'superior. Lung volumes appear are clear bilaterally. Heart and mediastinum normal limits in conto

# EXP03

## Customize the dataset class

In [8]:
from PIL import Image
class IUXRayDataset(Dataset):
    def __init__(self, image_dir, report_dir, tokenizer, max_length=100):
        self.image_dir = image_dir
        self.report_dir = report_dir
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = self.load_samples()

        # process image
        self.transform = transforms.Compose([
            transforms.Resize((224, 224)),
            transforms.ToTensor()
        ])

    def load_samples(self):
        print("===> Start loading samples from report and image...")
        samples = []
        count_img = 0
        count_match = 0
        count_missing_report = 0

        for img_name in tqdm(os.listdir(self.image_dir), desc="Parsing image-report pairs"):
            if not img_name.endswith('.png'):
                continue

            count_img += 1
            base_id = img_name.split('_')[0]  # CXR1005
            report_id = base_id.replace('CXR', '')  # 1005
            xml_path = os.path.join(self.report_dir, report_id + ".xml")

            if not os.path.exists(xml_path):
                count_missing_report += 1
                continue

            try:
                tree = ET.parse(xml_path)
                root = tree.getroot()
            except:
                continue

            # extract the FINDINGS 和 IMPRESSION
            findings, impression = "", ""
            for abstract in root.findall('.//AbstractText'):
                label = abstract.attrib.get('Label')
                if label == 'FINDINGS':
                    findings = abstract.text or ""
                if label == 'IMPRESSION':
                    impression = abstract.text or ""

          # structure the data
            fields = {
                "HEART": "N/A",
                "MEDIASTINUM": "N/A",
                "LUNGS": "N/A",
                "PLEURA": "N/A",
                "PNEUMOTHORAX": "N/A",
                "FINDINGS": findings,
                "IMPRESSION": impression
            }


            text = "\n".join([f"{k}: {v if v else 'N/A'}" for k, v in fields.items()])
            if not findings and not impression:
                continue  # empty report

            samples.append((img_name, text))
            count_match += 1

        print(f"✅ Total images scanned: {count_img}")
        print(f"✅ Total matched samples with report: {count_match}")
        print(f"⚠️  Total images missing report: {count_missing_report}")
        return samples




    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        img_name, text = self.samples[idx]
        img_path = os.path.join(self.image_dir, img_name)


        image = Image.open(img_path).convert("RGB")


        image = self.transform(image)

        tokens = self.tokenizer(
            text,
            padding='max_length',
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )

        return {
            'image': image,  # Tensor
            'input_ids': tokens.input_ids.squeeze(0),
            'attention_mask': tokens.attention_mask.squeeze(0)
        }

## Tokenizer

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("t5-small")  # 或 t5-base，或 gpt2


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

In [10]:
tokenizer.save_pretrained("./tokenizer")

('./tokenizer/tokenizer_config.json',
 './tokenizer/special_tokens_map.json',
 './tokenizer/spiece.model',
 './tokenizer/added_tokens.json',
 './tokenizer/tokenizer.json')

## Create Dataset and DataLoader

In [11]:
# 构建完整的 Dataset（所有数据）

full_dataset = IUXRayDataset(
    image_dir=image_dir,
    report_dir=report_dir,
    tokenizer=tokenizer,
    max_length=100  # 控制报告最长多少个 token
)


===> Start loading samples from report and image...


Parsing image-report pairs: 100%|██████████| 2065/2065 [01:20<00:00, 25.75it/s] 

✅ Total images scanned: 2063
✅ Total matched samples with report: 2054
⚠️  Total images missing report: 0


In [12]:

sample = full_dataset[0]

# image tensor shape
print("Image shape:", sample["image"].shape)

# Token IDs
print(" Token IDs:", sample["input_ids"].tolist())

# Decode  to original text
decoded_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
print(" Decoded Text:\n", decoded_text)



Image shape: torch.Size([3, 224, 224])
 Token IDs: [3, 6021, 8241, 10, 445, 87, 188, 3, 30296, 134, 25424, 6122, 10, 445, 87, 188, 301, 25158, 134, 10, 445, 87, 188, 3, 5329, 26296, 188, 10, 445, 87, 188, 276, 4171, 6122, 6951, 6299, 4763, 4, 10, 445, 87, 188, 377, 13885, 2365, 134, 10, 37, 16216, 8172, 7, 17, 10270, 19561, 19, 441, 1389, 6790, 21, 3179, 5, 465, 15949, 844, 13, 3, 26836, 16690, 5, 71, 3, 10379, 3676, 3, 7662, 83, 32, 51, 9, 19, 4313, 16, 8, 22586, 2663, 13, 8, 646, 1364, 3, 11846, 15, 5, 3104, 75, 3676, 25049, 3, 4, 1]
 Decoded Text:
 HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The cardiomediastinal silhouette is within normal limits for appearance. No focal areas of pulmonary consolidation. A calcified granuloma is identified in the peripheral aspect of the left lower lobe. Calcified lymph X


##Summary:
every image has a report, but not every report can find a matching image

# Training DataLoader & validation dataloader

In [13]:

import torch

train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = torch.utils.data.random_split(full_dataset, [train_size, val_size])

print(f"Training dataset: {len(train_dataset)}")
print(f"Testing Dataset: {len(val_dataset)}")

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

Training dataset: 1643
Testing Dataset: 411


## Encoder

In [14]:
from transformers import CLIPModel, CLIPProcessor


class EncoderCLIP(nn.Module):
    def __init__(self, embed_size, device):
        super(EncoderCLIP, self).__init__()
        self.device = device
        self.clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
        self.processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

        self.fc = nn.Linear(512, embed_size)
        self.bn = nn.BatchNorm1d(embed_size, momentum=0.01)

    def forward(self, images):
      inputs = self.processor(images=images, return_tensors="pt").to(self.device)


      image_features = self.clip_model.get_image_features(**inputs)  # [B, 512]
      projected = self.fc(image_features).unsqueeze(1)               # [B, 1, embed]

      return projected



## Decoder

In [15]:
import torch
import torch.nn as nn

class DecoderTransformer(nn.Module):
    def __init__(self, embed_size, vocab_size, num_layers=4, heads=4, ff_hidden=1024, dropout=0.3, max_len=100):
        super(DecoderTransformer, self).__init__()

        self.embed = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = PositionalEncoding(embed_size, dropout, max_len)

        decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_size,
            nhead=heads,
            dim_feedforward=ff_hidden,
            dropout=dropout,
            batch_first=True
        )

        self.transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(embed_size, vocab_size)

    def forward(self, image_features, captions):
        embeddings = self.embed(captions)
        embeddings = self.positional_encoding(embeddings)

        tgt_mask = self.generate_square_subsequent_mask(captions.size(1)).to(captions.device)

        output = self.transformer_decoder(
            tgt=embeddings,
            memory=image_features,
            tgt_mask=tgt_mask
        )

        return self.fc(output)


    def generate_square_subsequent_mask(self, sz):
        return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)


class PositionalEncoding(nn.Module):
    def __init__(self, embed_size, dropout=0.1, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, embed_size)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_size, 2).float() * (-torch.log(torch.tensor(10000.0)) / embed_size))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:, :x.size(1), :]
        return self.dropout(x)


In [16]:
class ImageCaptionModel(nn.Module):
    def __init__(self, embed_size, vocab_size, device):
        super(ImageCaptionModel, self).__init__()
        self.encoder = EncoderCLIP(embed_size, device)
        self.decoder = DecoderTransformer(embed_size, vocab_size)

    def forward(self, images, captions):
        image_features = self.encoder(images)  # [batch, embed]
        outputs = self.decoder(image_features, captions)
        return outputs


## Initialize the model

In [17]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = ImageCaptionModel(embed_size=512, vocab_size=tokenizer.vocab_size, device=device).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

## Train In loop

In [18]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

def train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=10, save_path="checkpoint.pth"):

    model.to(device)
    start_epoch = 0


    if os.path.exists(save_path):
        print(f"==> Found checkpoint at {save_path}, loading...")
        checkpoint = torch.load(save_path, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch'] + 1
        print(f"==> Resuming training from epoch {start_epoch}")
    model.train()

    for epoch in range(start_epoch,num_epochs):
        total_loss = 0
        loop = tqdm(train_loader, leave=True, dynamic_ncols=True)
        for batch in loop:
            images = batch['image'].to(device)
            captions = batch['input_ids'].to(device)

            optimizer.zero_grad()
            outputs = model(images, captions[:, :-1])
            loss = criterion(outputs.reshape(-1, outputs.size(-1)), captions[:, 1:].reshape(-1))
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            loop.set_description(f"Epoch [{epoch+1}/{num_epochs}]")
            loop.set_postfix(loss=loss.item())

        print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")


        checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': total_loss
        }


        os.makedirs(os.path.dirname(save_path), exist_ok=True)

        torch.save(checkpoint, save_path)
        print(f"Checkpoint saved at {save_path}")


In [19]:
train(model, train_loader, optimizer, criterion, tokenizer, device, num_epochs=120,save_path="/content/drive/MyDrive/checkpoint.pth")

Epoch [1/120]: 100%|██████████| 52/52 [21:45<00:00, 25.11s/it, loss=3.76]


Epoch 1, Loss: 329.4586
Checkpoint saved at /content/drive/MyDrive/checkpoint.pth


Epoch [2/120]: 100%|██████████| 52/52 [00:51<00:00,  1.01it/s, loss=2.16]


Epoch 2, Loss: 142.5535
Checkpoint saved at /content/drive/MyDrive/checkpoint.pth


Epoch [3/120]: 100%|██████████| 52/52 [00:48<00:00,  1.08it/s, loss=1.96]


...

Epoch 118, Loss: 12.4456
Checkpoint saved at /content/drive/MyDrive/checkpoint.pth


Epoch [119/120]: 100%|██████████| 52/52 [00:48<00:00,  1.08it/s, loss=0.23]


Epoch 119, Loss: 12.1998
Checkpoint saved at /content/drive/MyDrive/checkpoint.pth


Epoch [120/120]: 100%|██████████| 52/52 [01:24<00:00,  1.63s/it, loss=0.243]


Epoch 120, Loss: 12.1756
Checkpoint saved at /content/drive/MyDrive/checkpoint.pth


In [20]:

sample = train_dataset[0]

print("🖼️ Image shape:", sample["image"].shape)

# Token IDs
print("🧾 Token IDs:", sample["input_ids"].tolist())

decoded_text = tokenizer.decode(sample["input_ids"], skip_special_tokens=True)
print("📄 Decoded Text:\n", decoded_text)



🖼️ Image shape: torch.Size([3, 224, 224])
🧾 Token IDs: [3, 6021, 8241, 10, 445, 87, 188, 3, 30296, 134, 25424, 6122, 10, 445, 87, 188, 301, 25158, 134, 10, 445, 87, 188, 3, 5329, 26296, 188, 10, 445, 87, 188, 276, 4171, 6122, 6951, 6299, 4763, 4, 10, 445, 87, 188, 377, 13885, 2365, 134, 10, 37, 3, 4, 4, 4, 4, 6498, 3, 6848, 13, 851, 138, 11, 3, 12088, 2252, 9413, 7, 13, 8, 5738, 5, 26241, 3393, 3433, 3, 4, 4, 4, 4, 8, 3, 17, 21783, 226, 5, 37, 16216, 8172, 7, 17, 10270, 17643, 7, 33, 441, 1389, 6790, 5, 10035, 2157, 1208, 1]
📄 Decoded Text:
 HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The XXXX examination consists of frontal and lateral radiographs of the chest. External monitor leads XXXX the thorax. The cardiomediastinal contours are within normal limits. Pulmonary


## Evaluate

In [21]:
import torch
from tqdm import tqdm
from nltk.translate.bleu_score import corpus_bleu

def apply_repetition_penalty(logits, input_ids, penalty=1.2):

    for i in range(logits.size(0)):
        seen = set(input_ids[i].tolist())
        for token_id in seen:
            if logits[i, token_id] < 0:
                logits[i, token_id] *= penalty
            else:
                logits[i, token_id] /= penalty
    return logits

def evaluate(model, test_loader, tokenizer, device, max_length=100, num_samples=5,
             temperature=1.0, top_k=50, repetition_penalty=1.2):

    model.eval()
    model.to(device)

    eos_token_id = tokenizer.eos_token_id or tokenizer.sep_token_id or tokenizer.pad_token_id or 0
    bos_token_id = tokenizer.bos_token_id or tokenizer.cls_token_id or 0

    results = []

    with torch.no_grad():
        for batch in tqdm(test_loader):
            images = batch['image'].to(device)
            gt_reports = tokenizer.batch_decode(batch['input_ids'], skip_special_tokens=True)


            input_ids = torch.full((images.size(0), 1), bos_token_id, dtype=torch.long).to(device)

            finished = torch.zeros(images.size(0), dtype=torch.bool).to(device)

            for _ in range(max_length):
                outputs = model(images, input_ids)         # [B, T, vocab]
                logits = outputs[:, -1, :]
                logits = logits / temperature
                logits = apply_repetition_penalty(logits, input_ids, penalty=repetition_penalty)

                # Top-k sampling
                values, indices = torch.topk(logits, k=top_k, dim=-1)       # [B, K]
                probs = torch.softmax(values, dim=-1)                       # [B, K]
                sampled = torch.multinomial(probs, num_samples=1)          # [B, 1]
                next_token = indices.gather(-1, sampled)                   # [B, 1]

                input_ids = torch.cat([input_ids, next_token], dim=1)

                finished |= (next_token.squeeze() == eos_token_id)
                if finished.all():
                    break

            predictions = tokenizer.batch_decode(input_ids, skip_special_tokens=True)
            predictions = [p.strip() if p.strip() else "<EMPTY>" for p in predictions]
            gt_reports = [gt.strip() if gt.strip() else "<EMPTY>" for gt in gt_reports]

            for gt, pred in zip(gt_reports, predictions):
                results.append((gt, pred))


            if len(results) >= num_samples:
                results = results[:num_samples]
                break


    print("\n=== Sample Evaluation ===")
    for idx, (gt, pred) in enumerate(results):
        print(f"\n🔹 Sample {idx+1}")
        print(f"🟢 Ground Truth:\n{gt}")
        print(f"🔵 Prediction:\n{pred}")


    pred_tokens = [pred.split() for _, pred in results]
    gt_tokens = [[gt.split()] for gt, _ in results]
    bleu_score = corpus_bleu(gt_tokens, pred_tokens)
    print(f"\n🎯 BLEU Score: {bleu_score:.4f}")

    return results, bleu_score


In [22]:

# evaluate(model, val_loader, tokenizer, device, num_samples=5)
evaluate(
    model,
    val_loader,
    tokenizer,
    device,
    num_samples=5,
    temperature=1.0,
    top_k=50,
    repetition_penalty=1.2
)

  0%|          | 0/13 [01:03<?, ?it/s]


=== Sample Evaluation ===

🔹 Sample 1
🟢 Ground Truth:
HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The heart is normal in size and contour. There is no mediastinal widening. The lungs are clear bilaterally. No large pleural effusion or pneumothorax. The XXXX are intact. IM
🔵 Prediction:
ART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: There are low lung volumes. The lungs are otherwise clear without focal airspace consolidation. No pleural effusion or pneumothorax. Normal heart size and mediastinal contour. There is degenerative changes of the spine. IMPRESS

🔹 Sample 2
🟢 Ground Truth:
HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. Calcified granuloma is identified.
🔵 Prediction:
ART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMO

([('HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The heart is normal in size and contour. There is no mediastinal widening. The lungs are clear bilaterally. No large pleural effusion or pneumothorax. The XXXX are intact. IM',
   'ART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: There are low lung volumes. The lungs are otherwise clear without focal airspace consolidation. No pleural effusion or pneumothorax. Normal heart size and mediastinal contour. There is degenerative changes of the spine. IMPRESS'),
  ('HEART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: The heart size and pulmonary vascularity appear within normal limits. The lungs are free of focal airspace disease. No pleural effusion or pneumothorax is seen. Calcified granuloma is identified.',
   'ART: N/A MEDIASTINUM: N/A LUNGS: N/A PLEURA: N/A PNEUMOTHORAX: N/A FINDINGS: Frontal and lateral views of the chest with overlying external car